In [2]:
!pip install googlemaps
!pip install polyline
!pip install folium
!pip install scipy
!pip install pandas
!pip install geopy

import pandas as pd
import math
import scipy.optimize as opt
from geopy.geocoders import GoogleV3
import googlemaps
import polyline
import folium

# Enter your own API key
geolocator = GoogleV3(api_key='')
gmaps = googlemaps.Client(key='')    # Initialize the Google Maps client with your API key

In [3]:
# Define functions

# Function to get geocode for a location
def get_geocode(location):
    coords = geolocator.geocode(location)
    lat = round(coords.latitude, 4)
    lng = round(coords.longitude, 4)
    return lat, lng

# Great circle ("as crow flies") distance
def calc_dist_haversine(lat1, lng1, lat2, lng2):
  # Convert latitude and longitude from degrees to radians
  lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])

  a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lng2 - lng1) / 2) ** 2
  dist_haversine_miles = 3959 * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

  return dist_haversine_miles

# Function to calculate Haversine distance from a point to locations in dataframe
def calc_cost_haversine(coords, data):
    lat, lon = coords
    distances = []
    for _, row in data.iterrows():
        distances.append(calc_dist_haversine(lat, lon, row['Lat'], row['Lng']))

    cost = (data['F'] * data['W'] * pd.Series(distances)).sum()
    return cost

# Function to calculate travel distance from origin to each destination
def calc_dist_driving(lat1, lng1, lat2, lng2):

    # Request directions and get travel distance and travel time
    directions = gmaps.directions( (lat1, lng1), (lat2, lng2), mode='driving', units='imperial')

    # dist_travel_text = directions[0]['legs'][0]['distance']['text']
    # dist_travel_kms  = (directions[0]['legs'][0]['distance']['value']) // 1000
    dist_travel_mile = (directions[0]['legs'][0]['distance']['value']) // 1609.344

    # travel_time_text = directions[0]['legs'][0]['duration']['text']
    # travel_time_mins = (directions[0]['legs'][0]['duration']['value']) // 60

    return dist_travel_mile

# Function to calculate Haversine distance from a point to locations in dataframe
def calc_cost_driving(coords, data):
    lat, lon = coords
    distances = []
    for _, row in data.iterrows():
        distances.append(calc_dist_driving(lat, lon, row['Lat'], row['Lng']))

    cost = (data['F'] * data['W'] * pd.Series(distances)).sum()
    return cost


In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get the content of the Wikipedia page
def get_wiki_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# Function to extract the latitude and longitude
def get_lat_lon(neighborhood_url):
    try:
        response = requests.get(neighborhood_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        geo_tag = soup.find('span', {'class': 'geo'})
        if geo_tag:
            lat, lon = geo_tag.text.split('; ')
            return lat.strip(), lon.strip()
    except requests.RequestException:
        pass
    return None, None

# Function to extract the table and the corresponding neighborhood links
def parse_neighborhoods(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', {'class': 'wikitable sortable'})
    neighborhoods = []

    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        name = cols[0].text.strip()
        population = cols[1].text.strip().replace(',', '')
        link = cols[0].find('a')
        lat, lon = (None, None)
        if link and 'href' in link.attrs:
            lat, lon = get_lat_lon('https://en.wikipedia.org' + link.attrs['href'])
        neighborhoods.append({'Location': name+", Atlanta, GA", 'F': 1, 'W': int(population), 'Lat': lat, 'Lng': lon})
    
    return neighborhoods

# Main script
wiki_url = 'https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population'
content = get_wiki_page(wiki_url)
neighborhood_data = parse_neighborhoods(content)

# Convert to DataFrame and sort by population
df_neighborhoods = pd.DataFrame(neighborhood_data)
df_top_neighborhoods = df_neighborhoods.sort_values(by='W', ascending=False).head(10)

# Reset index and display the DataFrame
df_top_neighborhoods.reset_index(drop=True, inplace=True)
print(df_top_neighborhoods)


                              Location  F      W         Lat          Lng
0                 Midtown, Atlanta, GA  1  16569  33.7868014  -84.3795169
1                Downtown, Atlanta, GA  1  13411    33.75500    -84.39000
2         Old Fourth Ward, Atlanta, GA  1  10505      33.766      -84.372
3          North Buckhead, Atlanta, GA  1   8270   33.865252   -84.368863
4              Pine Hills, Atlanta, GA  1   8033      33.837      -84.352
5  Morningside/Lenox Park, Atlanta, GA  1   8030   33.797519   -84.358295
6       Virginia-Highland, Atlanta, GA  1   7800  33.7824000  -84.3543000
7              Grant Park, Atlanta, GA  1   6771      33.736      -84.371
8            Georgia Tech, Atlanta, GA  1   6607      33.776      -84.396
9                Kirkwood, Atlanta, GA  1   5897   33.752095   -84.323668


In [37]:
df = df_top_neighborhoods
df_top_neighborhoods['Lat'] = pd.to_numeric(df_top_neighborhoods['Lat'], errors='coerce')
df_top_neighborhoods['Lng'] = pd.to_numeric(df_top_neighborhoods['Lng'], errors='coerce')

# Get geocodes for each address in the DataFrame
# df[['Lat', 'Lng']] = df['Location'].apply(lambda x: pd.Series(get_geocode(x)))

### Step 1: First we find optimal point using Haversine distance
# Initial guess for the optimization algorithm (somewhere close to the average of latitudes and longitudes)
initial_guess = [df['Lat'].mean(), df['Lng'].mean()]

# Extract min and max values of Latitude and Longitude columns for bounds
bounds = [(df['Lat'].min() - 5, df['Lat'].max() + 5), (df['Lng'].min() - 5, df['Lng'].max() + 5)]

# Minimize the total distance function using scipy's minimize
result = opt.minimize(calc_cost_haversine, initial_guess, args=(df,), method='SLSQP', bounds=bounds)

# Get the location address
result_address = gmaps.reverse_geocode((result.x))[0]['formatted_address']

print(f"\nMinimum cost and corresponding optimal location (lat, long): \n$ {(result.fun):,.2f}; { (result.x).round(4) }\nAddress: {result_address}")



Minimum cost and corresponding optimal location (lat, long): 
$ 198,323.66; [ 33.7819 -84.3746]
Address: 10th St NE & Taft Ave NE, Atlanta, GA 30309, USA


In [38]:
### Step 2: Then we find driving distance using Google Maps Direction API

# Define ranges based on optimal point ± 0.5
diff = .02
ranges = (slice(int(result.x[0]) - diff, round(result.x[0], 0) + diff, 1),  slice(int(result.x[1]) - diff, round(result.x[1], 0) + diff, 1))

# Minimize the total distance function using scipy's integer minimizer
result_driving = opt.brute(calc_cost_driving, ranges, args=(df,), full_output=True, finish=None)
# Get the location address
result_driving_address = gmaps.reverse_geocode((result_driving[0]))[0]['formatted_address']

cost_haversine_driving = calc_cost_driving(result.x, df)
print(f"\nCost, Travel time using driving distance from point (lat, long) optimized using Haversine distance method : \n$ {cost_haversine_driving:,.2f}; { (result.x).round(4) }\nAddress: {result_address}")

print(f"\nMinimum cost, Travel time, and corresponding optimal location (lat, long): \n$ {result_driving[1]:,.2f}; { result_driving[0].tolist() }\nAddress: {result_driving_address}")


Cost, Travel time using driving distance from point (lat, long) optimized using Haversine distance method : 
$ 288,230.00; [ 33.7819 -84.3746]
Address: 10th St NE & Taft Ave NE, Atlanta, GA 30309, USA

Minimum cost, Travel time, and corresponding optimal location (lat, long): 
$ 2,624,476.00; [33.98, -84.02]
Address: 1255 Watercrest Cir, Lawrenceville, GA 30043, USA


In [39]:
### Code to print locations on map

# Initialize the map centered around the mean of all locations
m = folium.Map(location = initial_guess, zoom_start=5)

# Add markers for optimal locations
folium.Marker(result.x, popup=result_address, icon=folium.Icon(color='orange')).add_to(m)           # Point using Haversine distnace
folium.Marker(result_driving[0], popup=result_driving_address, icon=folium.Icon(color='green')).add_to(m)     # Point using Driving distance

# Add links between locations
# for i in range(len(df['Location']) - 1):
#     start = tuple(result_driving[0])
#     end   = (df['Lat'][i], df['Lng'][i])
#     folium.PolyLine([loc1, loc2], color="blue", weight=2.5, opacity=1).add_to(m)

# Draw driving paths between locations
for i in range(len(df['Location'])):
    start = tuple(result_driving[0])
    end   = (df['Lat'][i], df['Lng'][i])

    # Add markers for locations from DataFrame
    folium.Marker([df['Lat'][i], df['Lng'][i]], popup=df['Location'][i]).add_to(m)
    # Get directions using Google Maps Directions API from optimal point to all locations
    directions = gmaps.directions(start, end, mode="driving")
    # Extract polyline points from API response and add to Folium map
    points = polyline.decode(directions[0]['overview_polyline']['points'])
    # Add driving path on the map
    folium.PolyLine(locations=points, color='blue', weight=5).add_to(m)

# Save the map as an HTML file
#m.save('C:\Temp\map_with_locations_and_paths.html')

print(f"\nCost, Travel time using driving distance from point (lat, long) optimized using Haversine distance method : \n$ {cost_haversine_driving:,.2f}; { (result.x).round(4) }\nAddress: {result_address}")
print(f"\nMinimum cost, Travel time, and corresponding optimal location (lat, long): \n$ {result_driving[1]:,.2f}; { result_driving[0].tolist() }\nAddress: {result_driving_address}")



Cost, Travel time using driving distance from point (lat, long) optimized using Haversine distance method : 
$ 288,230.00; [ 33.7819 -84.3746]
Address: 10th St NE & Taft Ave NE, Atlanta, GA 30309, USA

Minimum cost, Travel time, and corresponding optimal location (lat, long): 
$ 2,624,476.00; [33.98, -84.02]
Address: 1255 Watercrest Cir, Lawrenceville, GA 30043, USA


In [40]:
display(m)